# LSTM for Fake News Detection

We will use **LSTM (Long Short-Term Memory)** networks to classify news as **Fake** or **Real**.  
Dataset: [Fake and Real News Dataset on Kaggle](https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset)

---

## 🔹 Limitation of Simple RNN
- RNNs process text step by step but struggle with **long sentences**.  
- They suffer from **vanishing gradients**, making it hard to remember earlier words.  
- Example: In *"The book I read yesterday was amazing"*, RNN may forget the link between *book* and *amazing*.  

---

## 🔹 Why LSTM?
- LSTM adds a **cell state** and **three gates** (forget, input, output).  
- This helps it **remember important info longer** and avoid vanishing gradients.  
- ✅ Works well for tasks like sentiment analysis, translation, and fake news detection.


In [1]:
import pandas as pd

# Load Fake and Real news CSVs
fake_df = pd.read_csv('dataset/fake-news/Fake.csv')
true_df = pd.read_csv('dataset/fake-news/True.csv')


In [6]:
# Add label column: 0 = fake, 1 = real
fake_df['label'] = 0
true_df['label'] = 1

In [3]:
# Combine both dataframes
df = pd.concat([fake_df, true_df], ignore_index=True)

# Shuffle the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)


In [4]:
# Keep only 'text' and 'label' columns
df = df[['text', 'label']]

# Check first few rows
df.head()

,text,label
0,"21st Century Wire says Ben Stein, reputable pr...",0
1,WASHINGTON (Reuters) - U.S. President Donald T...,1
2,(Reuters) - Puerto Rico Governor Ricardo Rosse...,1
3,"On Monday, Donald Trump once again embarrassed...",0
4,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",1


In [7]:
# Prepare Features and Labels
# Independent feature
X = df['text']

# Dependent feature
y = df['label']

# Check shapes
print("Number of samples:", len(X))
print("Number of labels:", len(y))


Number of samples: 44898
Number of labels: 44898


In [8]:
# Text Preprocessing for LSTM
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Vocabulary size
voc_size = 5000

# One-hot encode each sentence
onehot_repr = [one_hot(words, voc_size) for words in X]

# Set max sentence length for padding
sent_length = 20

# Pad sequences to have the same length
embedded_docs = pad_sequences(onehot_repr, padding='post', maxlen=sent_length)

# Check first padded sentence
print(embedded_docs[0])


[2001 4086  203 3468  314 3346 4451 4166 1068 3468 2836   65 4058 1981
 4336  792 2352 4232 4279 2769]


In [9]:
# Prepare Data for Model
import numpy as np

# Convert to numpy arrays
X_final = np.array(embedded_docs)
y_final = np.array(y)

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)


Training data shape: (30081, 20)
Testing data shape: (14817, 20)


In [11]:
# Build the model
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# Compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 👇 Force build with expected input shape
model.build(input_shape=(None, sent_length))

# Summary
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 20, 40)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100)                 │          56,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)


Epoch 1/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 21s 36ms/step - accuracy: 0.9104 - loss: 0.2103 - val_accuracy: 0.9519 - val_loss: 0.1243
Epoch 2/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 18s 38ms/step - accuracy: 0.9586 - loss: 0.1086 - val_accuracy: 0.9490 - val_loss: 0.1365
Epoch 3/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 16s 34ms/step - accuracy: 0.9683 - loss: 0.0869 - val_accuracy: 0.9548 - val_loss: 0.1266
Epoch 4/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step - accuracy: 0.9750 - loss: 0.0709 - val_accuracy: 0.9566 - val_loss: 0.1280
Epoch 5/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 17s 35ms/step - accuracy: 0.9808 - loss: 0.0555 - val_accuracy: 0.9557 - val_loss: 0.1329
Epoch 6/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 16s 34ms/step - accuracy: 0.9860 - loss: 0.0411 - val_accuracy: 0.9562 - val_loss: 0.1476
Epoch 7/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 15s 32ms/step - accuracy: 0.9892 - loss: 0.0331 - val_accuracy: 0.9571 - val_loss: 0.1517
Epoch 8/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 15s 32ms/step - accuracy: 0.9915 - loss: 0.0254 - 

In [14]:
# Predict on test data
y_pred = model.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)  # threshold 0.5

# Metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nAccuracy Score:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


464/464 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step
Confusion Matrix:
 [[7506  248]
 [ 460 6603]]

Accuracy Score: 0.9522170479854222

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.97      0.95      7754
           1       0.96      0.93      0.95      7063

    accuracy                           0.95     14817
   macro avg       0.95      0.95      0.95     14817
weighted avg       0.95      0.95      0.95     14817



In [18]:
import random
import numpy as np

# Pick 5 random samples from test set
indices = random.sample(range(len(X_test)), 5)

for i in indices:
    # Get original text from X (just a string, not DataFrame with 'title')
    text = X.iloc[i]   # Directly use iloc because it's a Series
    true_label = y.iloc[i]
    
    # Predict using model
    pred_prob = model.predict(X_final[i].reshape(1, -1))[0][0]
    pred_label = 1 if pred_prob > 0.5 else 0
    
    print("="*80)
    print(f"📰 News: {text[:300]}...")   # show first 300 chars
    print(f"✅ Actual: {'True' if true_label==1 else 'Fake'}")
    print(f" Predicted: {'True' if pred_label==1 else 'Fake'} (prob: {pred_prob:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
📰 News: CLEARWATER, Fla. (Reuters) - While U.S. President Donald Trump tried to win over skeptics in Congress on Tuesday, he never had to worry about losing the faithful at the Quaker Steak and Lube. About two dozen fans of the 45th president gathered at the bar and restaurant in Clearwater, Florida, still ...
✅ Actual: True
 Predicted: True (prob: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
📰 News: NEW YORK (Reuters) - Republican presidential candidate Ben Carson’s comments that President Barack Obama was “raised white” prompted backlash on social media on Tuesday, with many users criticizing the neurosurgeon for stereotyping the experience of African-Americans. Carson, who is black and grew u...
✅ Actual: True
 Predicted: True (prob: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
📰 News: WASHINGTON (Reuters) - For the moment, U.S. President Donald Trump is going it alone. After weeks of seeing his agenda imperiled by Republican divisions and infight

### Key Points:

- LSTM can **handle long sequences** better than vanilla RNNs.
- Our model achieved a **reasonable accuracy** on the Fake/Real News dataset.
- Next steps:
  - Use **larger embedding size** or **pre-trained embeddings** like GloVe.
  - Try **bidirectional LSTM** for better context understanding.
  - Experiment with **more epochs** or **different batch sizes**.
